In [1]:
import pandas as pd
import numpy as np
import re
import csv

from bs4 import BeautifulSoup
import requests
import time, os 
import random
from fake_useragent import UserAgent

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

ua = UserAgent()


In [2]:
movies = pd.read_csv('./movies_2021_2017.csv')

In [3]:
movies.shape

(4452, 12)

In [4]:
movies.tail()

,Unnamed: 0,title,movie_id,movie_imdb_link,certificate,runtime_in_mins,genre,imdb_rating,number_of_votes,metascore,top_director,release_year
4447,4447,WHAT IF? A (Fan-Made) 'Life is Strange' Story,tt7732486,https://www.imdb.com/title/tt7732486/?ref_=adv...,[],127,"Drama, Mystery",5.1,204,[],See full summary,2017
4448,4448,Created Equal,tt5873100,https://www.imdb.com/title/tt5873100/?ref_=adv...,TV-PG,91,"Drama, Thriller",5.5,204,[],Bill Duke,2017
4449,4449,Kate Can't Swim,tt5752904,https://www.imdb.com/title/tt5752904/?ref_=adv...,[],90,Drama,5.5,201,[],Josh Helman,2017
4450,4450,Blue Hollywood,tt5588118,https://www.imdb.com/title/tt5588118/?ref_=adv...,[],82,"Comedy, Drama, Romance",7.3,200,[],See full summary,2017
4451,4451,A Closer Walk with Thee,tt4943620,https://www.imdb.com/title/tt4943620/?ref_=adv...,Unrated,89,"Drama, Horror",3.8,200,[],John C. Clark,2017


In [5]:
movies = movies.drop(['Unnamed: 0'], axis=1) 
movies.head()

,title,movie_id,movie_imdb_link,certificate,runtime_in_mins,genre,imdb_rating,number_of_votes,metascore,top_director,release_year
0,Spider-Man: No Way Home,tt10872600,https://www.imdb.com/title/tt10872600/?ref_=ad...,PG-13,148,"Action, Adventure, Fantasy",8.3,739086,71,Jon Watts,2021
1,Dune,tt1160419,https://www.imdb.com/title/tt1160419/?ref_=adv...,PG-13,155,"Action, Adventure, Drama",8.0,621069,74,Denis Villeneuve,2021
2,Don't Look Up,tt11286314,https://www.imdb.com/title/tt11286314/?ref_=ad...,R,138,"Comedy, Drama, Sci-Fi",7.2,529981,49,Adam McKay,2021
3,No Time to Die,tt2382320,https://www.imdb.com/title/tt2382320/?ref_=adv...,PG-13,163,"Action, Adventure, Thriller",7.3,393930,68,Cary Joji Fukunaga,2021
4,Zack Snyder's Justice League,tt12361974,https://www.imdb.com/title/tt12361974/?ref_=ad...,R,242,"Action, Adventure, Fantasy",8.0,393504,54,Zack Snyder,2021


# A df with the awards link


In [145]:
movie_awards_link = movies[['title', 'movie_id', 'movie_imdb_link']]

In [146]:
movie_awards_link['awards_link'] = ''

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_53513/3436540390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_awards_link['awards_link'] = ''


In [147]:
movie_awards_link.head()

,title,movie_id,movie_imdb_link,awards_link
0,Spider-Man: No Way Home,tt10872600,https://www.imdb.com/title/tt10872600/?ref_=ad...,
1,Dune,tt1160419,https://www.imdb.com/title/tt1160419/?ref_=adv...,
2,Don't Look Up,tt11286314,https://www.imdb.com/title/tt11286314/?ref_=ad...,
3,No Time to Die,tt2382320,https://www.imdb.com/title/tt2382320/?ref_=adv...,
4,Zack Snyder's Justice League,tt12361974,https://www.imdb.com/title/tt12361974/?ref_=ad...,


In [148]:
for i in movie_awards_link.index:
    awards_link_temp = movie_awards_link['movie_imdb_link'][i].split('/?ref_=adv_li_tt')[0]
    movie_awards_link['awards_link'][i] = awards_link_temp + '/awards/?ref_=tt_awd'
    

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_53513/2932944961.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_awards_link['awards_link'][i] = awards_link_temp + '/awards/?ref_=tt_awd'


In [149]:
movie_awards_link.head()


,title,movie_id,movie_imdb_link,awards_link
0,Spider-Man: No Way Home,tt10872600,https://www.imdb.com/title/tt10872600/?ref_=ad...,https://www.imdb.com/title/tt10872600/awards/?...
1,Dune,tt1160419,https://www.imdb.com/title/tt1160419/?ref_=adv...,https://www.imdb.com/title/tt1160419/awards/?r...
2,Don't Look Up,tt11286314,https://www.imdb.com/title/tt11286314/?ref_=ad...,https://www.imdb.com/title/tt11286314/awards/?...
3,No Time to Die,tt2382320,https://www.imdb.com/title/tt2382320/?ref_=adv...,https://www.imdb.com/title/tt2382320/awards/?r...
4,Zack Snyder's Justice League,tt12361974,https://www.imdb.com/title/tt12361974/?ref_=ad...,https://www.imdb.com/title/tt12361974/awards/?...


# Scraping the awards information

In [25]:
def scrape_awards(df):
    
    movie_awards_dict = {}  #initializing an empty dictionary to fill with information

    for i in df.index:
        title = df['title'][i]
        movie_id = df['movie_id'][i]
        awards_link = df['awards_link'][i]
        
        user_agent = {'User-agent': ua.random} # generates a random user agent.
        response = requests.get(awards_link,headers = user_agent).text
        soup = BeautifulSoup(response, "html.parser")

        # awards and nominations:
        try:
            awards_total = soup.select('#main > div:nth-child(1) > div > div.header > div > div')[0].get_text()
        except:  
            awards_total = 0
        
        time.sleep(.5+2*random.random()) # random time between requests.
        
        movie_awards_dict[df.index[i]] = [title]+[movie_id]+[awards_link]+[awards_total]

    # define headers
    headers = ['title','movie_id','awards_link', 'awards_total']
    
    # turn into a dataframe    
    movie_awards_dataframe = pd.DataFrame(movie_awards_dict).T
    movie_awards_dataframe.columns = headers
    
    #save the dataframe
    #movie_awards_dataframe.to_csv('awards_2022.csv') # CHANGE THE YEAR ACC TO YOUR INTEREST
    
    return movie_awards_dataframe


In [214]:
awards_df = scrape_awards(movie_awards_link)

In [215]:
awards_df.head()

,title,movie_id,awards_link,awards_total
0,Spider-Man: No Way Home,tt10872600,https://www.imdb.com/title/tt10872600/awards/?...,Showing all 34 wins and 66 nominations
1,Dune,tt1160419,https://www.imdb.com/title/tt1160419/awards/?r...,Showing all 164 wins and 280 nominations
2,Don't Look Up,tt11286314,https://www.imdb.com/title/tt11286314/awards/?...,Showing all 19 wins and 88 nominations
3,No Time to Die,tt2382320,https://www.imdb.com/title/tt2382320/awards/?r...,Showing all 35 wins and 69 nominations
4,Zack Snyder's Justice League,tt12361974,https://www.imdb.com/title/tt12361974/awards/?...,Showing all 1 win and 6 nominations


# Repeating everything for another dataframe

In [218]:
movies_2016_2015 = pd.read_csv('./movies_2016_2015.csv')

In [219]:
movies_2016_2015 = movies_2016_2015.drop(['Unnamed: 0'], axis=1) 
movies_2016_2015.tail(3)

,title,movie_id,movie_imdb_link,certificate,runtime_in_mins,genre,imdb_rating,number_of_votes,metascore,top_director,release_year
1885,The Haunting at Woodland Hills,tt1523575,https://www.imdb.com/title/tt1523575/?ref_=adv...,[],84,"Drama, Horror, Thriller",3.4,201,[],Jeremy Casper,2016
1886,Volumes of Blood: Horror Stories,tt6211920,https://www.imdb.com/title/tt6211920/?ref_=adv...,[],118,Horror,3.9,200,[],See full summary,2016
1887,Game of Aces,tt3687118,https://www.imdb.com/title/tt3687118/?ref_=adv...,R,97,"Action, Adventure, War",4.2,200,[],Damien Lay,2016


In [220]:
movie_awards_link_2 = movies_2016_2015[['title', 'movie_id', 'movie_imdb_link']]

In [221]:
movie_awards_link_2['awards_link'] = ''

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_53513/1736227159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_awards_link_2['awards_link'] = ''


In [222]:
for i in movie_awards_link_2.index:
    awards_link_temp_2 = movie_awards_link_2['movie_imdb_link'][i].split('/?ref_=adv_li_tt')[0]
    movie_awards_link_2['awards_link'][i] = awards_link_temp_2 + '/awards/?ref_=tt_awd'
    

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_53513/3521889068.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_awards_link_2['awards_link'][i] = awards_link_temp_2 + '/awards/?ref_=tt_awd'


In [223]:
movie_awards_link_2.head()


,title,movie_id,movie_imdb_link,awards_link
0,Star Wars: Episode VII - The Force Awakens,tt2488496,https://www.imdb.com/title/tt2488496/?ref_=adv...,https://www.imdb.com/title/tt2488496/awards/?r...
1,Avengers: Age of Ultron,tt2395427,https://www.imdb.com/title/tt2395427/?ref_=adv...,https://www.imdb.com/title/tt2395427/awards/?r...
2,The Martian,tt3659388,https://www.imdb.com/title/tt3659388/?ref_=adv...,https://www.imdb.com/title/tt3659388/awards/?r...
3,The Revenant,tt1663202,https://www.imdb.com/title/tt1663202/?ref_=adv...,https://www.imdb.com/title/tt1663202/awards/?r...
4,Inside Out,tt2096673,https://www.imdb.com/title/tt2096673/?ref_=adv...,https://www.imdb.com/title/tt2096673/awards/?r...


In [225]:
awards_2016_2015_df = scrape_awards(movie_awards_link_2)

In [226]:
awards_2016_2015_df.head(3)

,title,movie_id,awards_link,awards_total
0,Star Wars: Episode VII - The Force Awakens,tt2488496,https://www.imdb.com/title/tt2488496/awards/?r...,Showing all 62 wins and 139 nominations
1,Avengers: Age of Ultron,tt2395427,https://www.imdb.com/title/tt2395427/awards/?r...,Showing all 8 wins and 51 nominations
2,The Martian,tt3659388,https://www.imdb.com/title/tt3659388/awards/?r...,Showing all 40 wins and 200 nominations


# Validation data (year 2022)

In [13]:
movies_2022 = pd.read_csv('./movies_2022.csv')

In [14]:
movies_2022.tail()

,Unnamed: 0,movie_id,movie_imdb_link,certificate,runtime_in_mins,genre,imdb_rating,number_of_votes,metascore,top_director,release_year
472,Call for Peace,tt19511880,https://www.imdb.com/title/tt19511880/?ref_=ad...,[],[],Reality-TV,9.1,204.0,[],Diane Paloma Eskenazi,2022
473,Do Not Disturb,tt10696116,https://www.imdb.com/title/tt10696116/?ref_=ad...,[],92,Horror,7.9,204.0,[],John Ainslie,2022
474,Matilda the Musical,tt3447590,https://www.imdb.com/title/tt3447590/?ref_=adv...,PG,117,"Comedy, Drama, Family",6.1,201.0,67,Matthew Warchus,2022
475,The Haunting of the Murder House,tt20861742,https://www.imdb.com/title/tt20861742/?ref_=ad...,[],78,Horror,4.5,200.0,[],Brendan Rudnicki,2022
476,Project Legion,tt12885770,https://www.imdb.com/title/tt12885770/?ref_=ad...,R,90,"Horror, Sci-Fi, Thriller",2.2,200.0,[],Lance Kawas,2022


In [15]:
movies_2022 = movies_2022.reset_index()
movies_2022 = movies_2022.rename(columns = {'Unnamed: 0':'title'})


In [16]:
movies_2022.shape

(477, 12)

In [17]:
movies_2022.head()

,index,title,movie_id,movie_imdb_link,certificate,runtime_in_mins,genre,imdb_rating,number_of_votes,metascore,top_director,release_year
0,0,The Batman,tt1877830,https://www.imdb.com/title/tt1877830/?ref_=adv...,PG-13,176,"Action, Crime, Drama",7.9,613958.0,72,Matt Reeves,2022
1,1,Top Gun: Maverick,tt1745960,https://www.imdb.com/title/tt1745960/?ref_=adv...,PG-13,130,"Action, Drama",8.4,413022.0,78,Joseph Kosinski,2022
2,2,Doctor Strange in the Multiverse of Madness,tt9419884,https://www.imdb.com/title/tt9419884/?ref_=adv...,PG-13,126,"Action, Adventure, Fantasy",6.9,398982.0,60,Sam Raimi,2022
3,3,Thor: Love and Thunder,tt10648342,https://www.imdb.com/title/tt10648342/?ref_=ad...,PG-13,118,"Action, Adventure, Comedy",6.4,296865.0,57,Taika Waititi,2022
4,4,Everything Everywhere All at Once,tt6710474,https://www.imdb.com/title/tt6710474/?ref_=adv...,R,139,"Action, Adventure, Comedy",8.1,228555.0,81,Dan Kwan,2022


In [19]:
movies_2022 = movies_2022.drop(['index'], axis=1) 
movies_2022.head()

,title,movie_id,movie_imdb_link,certificate,runtime_in_mins,genre,imdb_rating,number_of_votes,metascore,top_director,release_year
0,The Batman,tt1877830,https://www.imdb.com/title/tt1877830/?ref_=adv...,PG-13,176,"Action, Crime, Drama",7.9,613958.0,72,Matt Reeves,2022
1,Top Gun: Maverick,tt1745960,https://www.imdb.com/title/tt1745960/?ref_=adv...,PG-13,130,"Action, Drama",8.4,413022.0,78,Joseph Kosinski,2022
2,Doctor Strange in the Multiverse of Madness,tt9419884,https://www.imdb.com/title/tt9419884/?ref_=adv...,PG-13,126,"Action, Adventure, Fantasy",6.9,398982.0,60,Sam Raimi,2022
3,Thor: Love and Thunder,tt10648342,https://www.imdb.com/title/tt10648342/?ref_=ad...,PG-13,118,"Action, Adventure, Comedy",6.4,296865.0,57,Taika Waititi,2022
4,Everything Everywhere All at Once,tt6710474,https://www.imdb.com/title/tt6710474/?ref_=adv...,R,139,"Action, Adventure, Comedy",8.1,228555.0,81,Dan Kwan,2022


In [20]:
movie_awards_link_2022 = movies_2022[['title', 'movie_id', 'movie_imdb_link']]

In [21]:
movie_awards_link_2022['awards_link'] = ''

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_61402/3446922147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_awards_link_2022['awards_link'] = ''


In [22]:
movie_awards_link_2022.head()

,title,movie_id,movie_imdb_link,awards_link
0,The Batman,tt1877830,https://www.imdb.com/title/tt1877830/?ref_=adv...,
1,Top Gun: Maverick,tt1745960,https://www.imdb.com/title/tt1745960/?ref_=adv...,
2,Doctor Strange in the Multiverse of Madness,tt9419884,https://www.imdb.com/title/tt9419884/?ref_=adv...,
3,Thor: Love and Thunder,tt10648342,https://www.imdb.com/title/tt10648342/?ref_=ad...,
4,Everything Everywhere All at Once,tt6710474,https://www.imdb.com/title/tt6710474/?ref_=adv...,


In [23]:
for i in movie_awards_link_2022.index:
    awards_link_temp_2022 = movie_awards_link_2022['movie_imdb_link'][i].split('/?ref_=adv_li_tt')[0]
    movie_awards_link_2022['awards_link'][i] = awards_link_temp_2022 + '/awards/?ref_=tt_awd'
    

/var/folders/33/p_3l01b14g96rn22vzwly2g00000gn/T/ipykernel_61402/3228027479.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_awards_link_2022['awards_link'][i] = awards_link_temp_2022 + '/awards/?ref_=tt_awd'


In [24]:
movie_awards_link_2022.head()


,title,movie_id,movie_imdb_link,awards_link
0,The Batman,tt1877830,https://www.imdb.com/title/tt1877830/?ref_=adv...,https://www.imdb.com/title/tt1877830/awards/?r...
1,Top Gun: Maverick,tt1745960,https://www.imdb.com/title/tt1745960/?ref_=adv...,https://www.imdb.com/title/tt1745960/awards/?r...
2,Doctor Strange in the Multiverse of Madness,tt9419884,https://www.imdb.com/title/tt9419884/?ref_=adv...,https://www.imdb.com/title/tt9419884/awards/?r...
3,Thor: Love and Thunder,tt10648342,https://www.imdb.com/title/tt10648342/?ref_=ad...,https://www.imdb.com/title/tt10648342/awards/?...
4,Everything Everywhere All at Once,tt6710474,https://www.imdb.com/title/tt6710474/?ref_=adv...,https://www.imdb.com/title/tt6710474/awards/?r...


In [26]:
awards_2022_df = scrape_awards(movie_awards_link_2022)

In [27]:
awards_2022_df.head()

,title,movie_id,awards_link,awards_total
0,The Batman,tt1877830,https://www.imdb.com/title/tt1877830/awards/?r...,Showing all 2 wins and 27 nominations
1,Top Gun: Maverick,tt1745960,https://www.imdb.com/title/tt1745960/awards/?r...,Showing all 2 wins and 10 nominations
2,Doctor Strange in the Multiverse of Madness,tt9419884,https://www.imdb.com/title/tt9419884/awards/?r...,Showing all 0 wins and 8 nominations
3,Thor: Love and Thunder,tt10648342,https://www.imdb.com/title/tt10648342/awards/?...,Showing all 0 wins and 6 nominations
4,Everything Everywhere All at Once,tt6710474,https://www.imdb.com/title/tt6710474/awards/?r...,Showing all 10 wins and 14 nominations
